# Scrap from coworker.com

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import os

# Setup Selenium WebDriver
chromedriver_path = "/workspaces/Coworking/chromedriver-linux64/chromedriver"
chrome_options = Options()
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--headless=false")  # Run with visible browser

service = Service(chromedriver_path)
driver = webdriver.Chrome(service=service, options=chrome_options)

# Open coworking page
city = "madrid"
base_url = f"https://www.coworker.com/spain/{city}?view=list"
driver.get(base_url)

time.sleep(5)  # Wait for page to load

# Save page source for manual inspection
with open("/workspaces/Coworking/page_source.html", "w", encoding="utf-8") as f:
    f.write(driver.page_source)

# Print coworking space elements
try:
    coworking_blocks = driver.find_elements(By.CSS_SELECTOR, "div.SearchResult_container__0EI6G")  # Adjust if needed
    print(f"Found {len(coworking_blocks)} coworking spaces.")

    for i, block in enumerate(coworking_blocks[:5]):  # Print first 5 elements
        print(f"🔹 Block {i+1} HTML:")
        print(block.get_attribute("outerHTML"))
        print("=" * 50)

except Exception as e:
    print("Error finding coworking blocks:", e)

driver.quit()


Found 0 coworking spaces.


In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
import pandas as pd
import os

def scrape_coworker_data(driver, city="madrid"):
    base_url = f"https://www.coworker.com/spain/{city}?view=list"
    driver.get(base_url)

    time.sleep(5)  # Wait for content to load

    # ✅ Save page source for debugging
    with open(f"/workspaces/Coworking/page_source_{city}.html", "w", encoding="utf-8") as f:
        f.write(driver.page_source)

    # ✅ Print page title to confirm loading
    print(f"Page title: {driver.title}")

    coworkings, addresses, amenities, prices = [], [], [], []

    while True:
        wait = WebDriverWait(driver, 10)
        try:
            coworking_blocks = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.SearchResult_container__0EI6G")))
            print(f"Found {len(coworking_blocks)} coworking spaces on this page.")
        except Exception as e:
            print("❌ Error finding coworking blocks:", e)
            break

        for block in coworking_blocks:
            try:
                name = block.find_element(By.CSS_SELECTOR, "p.SearchResult_container__content__title__fSW47").text
                address = block.find_element(By.CSS_SELECTOR, "span.Transportation_name__CkTxv").text
                amenities_str = ", ".join([a.text for a in block.find_elements(By.CSS_SELECTOR, "div.amenities li")]) or "N/A"
                price = block.find_element(By.CSS_SELECTOR, "div.Prices_container__-8jBv p").text or "N/A"

                coworkings.append(name)
                addresses.append(address)
                amenities.append(amenities_str)
                prices.append(price)

            except Exception as e:
                print(f"⚠️ Error scraping a coworking space: {e}")

        try:
            next_button = driver.find_element(By.CSS_SELECTOR, "button.Pagination_page_link__4IZxn")
            next_button.click()
            time.sleep(5)
        except:
            print("✅ No more pages. Scraping complete.")
            break

    df = pd.DataFrame({
        "Coworking Name": coworkings,
        "Address": addresses,
        "Amenities": amenities,
        "Price (per month)": prices
    })

    print(df.head(), f"Total rows: {df.shape[0]}")

    save_path = "/workspaces/Coworking/src/results/Madrid/"
    os.makedirs(save_path, exist_ok=True)
    csv_filename = os.path.join(save_path, f"coworkings_{city}.csv")
    df.to_csv(csv_filename, index=False)

    print(f"✅ CSV file saved at {csv_filename}")

# ---------- Main Execution ----------
if __name__ == "__main__":
    chromedriver_path = "/workspaces/Coworking/chromedriver-linux64/chromedriver"

    if not os.path.exists(chromedriver_path):
        raise FileNotFoundError(f"ChromeDriver not found at {chromedriver_path}")

    chrome_options = Options()
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--headless=false")  # ✅ Run with a visible browser for debugging

    service = Service(chromedriver_path)
    driver = webdriver.Chrome(service=service, options=chrome_options)

    print("🚀 Starting web scraping...")
    scrape_coworker_data(driver, city="madrid")

    driver.quit()


🚀 Starting web scraping...
Page title: Just a moment...
❌ Error finding coworking blocks: Message: 

Empty DataFrame
Columns: [Coworking Name, Address, Amenities, Price (per month)]
Index: [] Total rows: 0
✅ CSV file saved at /workspaces/Coworking/src/results/Madrid/coworkings_madrid.csv


In [2]:
df.head()

NameError: name 'df' is not defined